# Prevedere il crime rate

## Caricamento Librerie
Per prima cosa carichiamo le librerie per effettuare operazioni sui dati

*   NumPy per creare e operare su array a N dimensioni
*   pandas per caricare e manipolare dati tabulari
*   matplotlib per creare grafici

Importiamo le librerie usando i loro alias convenzionali

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

## Caricamento dei dati
I dataset presi sono satti scricati dal sito uffucuale delle nazioni unite (da completare)

In [ ]:
import os.path
file = "crime_rate.csv"